<a href="https://colab.research.google.com/github/desai-nitin/CDAC_2018/blob/master/ElectricityTheftDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Electricity Theft Detection CDAC-Final Project**

## ***DataPreparation***

In [1]:
import numpy as np
import pandas as pd
import sklearn
from tensorflow import keras
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=---

Enter your authorization code:
··········
Mounted at /content/gdrive


**Importing Data**

---



In [0]:
df=pd.read_csv('/content/gdrive/My Drive/CDAC_18/Project/Dataset/data.csv')

**Sample Creation**

---



In [0]:
sample_df=df.sample(n=100)

sample_df.to_csv(path_or_buf="/content/gdrive/My Drive/CDAC_18/Project/Dataset/sampleData.csv",header=list(df.columns.values))


Data Cleaning
---
***Working on Sample***



In [2]:
#df = pd.DataFrame(df.iloc[:,1:].values)     # drop column 1 coz python generates index column on dataframe
dfs=pd.read_csv("/content/gdrive/My Drive/CDAC_18/Project/Dataset/sampleData.csv")
dfs = dfs.dropna(axis='columns', how='all')
dfs = dfs.drop("Unnamed: 0", axis=1)

df_iloc = dfs.iloc[:,:].values     # seperate dataframe to perform processing .. keeping df as it is to save headers

from sklearn.preprocessing import Imputer,LabelEncoder

consConvert=LabelEncoder()
df_iloc[:,0]=consConvert.fit_transform(df_iloc[:,0].astype(str))    # Categorical CONS_NO to NUMERIC

df_iloc = pd.DataFrame(df_iloc)

df_y = pd.DataFrame(df_iloc.iloc[:,:2].values)   # consider first 2 columns 

df_x = pd.DataFrame(df_iloc.iloc[:,2:].values)   # consider rest all columns for moving average and normalization

imputer=Imputer(missing_values="NaN",strategy="mean",axis=1)   # replace NaN with mean

df_x=pd.DataFrame(imputer.fit_transform(df_x))

df_x = (df_x-min(df_x))/(max(df_x)-min(df_x))   # normalization of df_x which contains all numeric values of 1035 days

df_iloc = [df_y,df_x]       # concatenate df_x and df_y into df_iloc 
df_iloc = pd.concat(df_iloc, axis=1,join='inner')

# save dataframe as csv with headers of actual df
df_iloc.to_csv(path_or_buf="/content/gdrive/My Drive/CDAC_18/Project/Dataset/FinalData.csv", header=list(dfs.columns.values), index=False)

#need to add moving avg imputation for NaN values

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


### Test

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/CDAC_18/Project/Dataset/FinalData.csv").head() #read sample dataset


In [19]:
test_df=pd.DataFrame(df.iloc[:,2:].values,columns=df.columns.values[2:])
print(test_df)
test_df=test_df.sort_values(by=1,axis=1)
print(test_df)

   2014/1/1  2014/1/10  2014/1/11  2014/1/12  2014/1/13  2014/1/14  2014/1/15  \
0  0.005475   0.005475   0.005475   0.005475   0.005475   0.005475   0.005475   
1  0.010136   0.010136   0.010136   0.010136   0.010136   0.010136   0.010136   
2  0.000000   0.054205   0.064990   0.064244   0.055727   0.000000   0.000000   
3  0.008713   0.008713   0.008713   0.008713   0.008713   0.008713   0.008713   
4  0.005446   0.005533   0.004903   0.006192   0.005262   0.005455   0.005271   

   2014/1/16  2014/1/17  2014/1/18    ...     2016/9/28  2016/9/29  2016/9/3  \
0   0.005475   0.005475   0.005475    ...      0.006919   0.006318  0.007335   
1   0.010136   0.010136   0.010136    ...      0.007742   0.009050  0.009147   
2   0.000000   0.000000   0.000000    ...      0.001153   0.001134  0.001521   
3   0.008713   0.008713   0.008713    ...      0.001579   0.001725  0.001938   
4   0.000000   0.005417   0.005707    ...      0.003527   0.009264  0.000659   

   2016/9/30  2016/9/4  2016/9/5

## **Implementing Model for sample**

---



Wide implementation

In [0]:
test_df=pd.DataFrame(df.iloc[:,2:].values).head()
print(test_df)

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/CDAC_18/Project/Dataset/FinalData.csv") #read sample dataset

df_y = df.iloc[:,1].values
df_x = df.iloc[:,2:-4].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y,train_size=0.8)

layers = keras.layers
alpha=60
# wide model 
wideInputs = layers.Input(shape=(1029,))
#fullyConnectedLayer = layers.Flatten()(wideInputs)
fullyConnectedLayer = layers.Dense(alpha,activation='relu',input_shape=(1029,))(wideInputs)
wideOutputs = layers.Dense(1, activation = 'sigmoid')(fullyConnectedLayer)
wideModel = keras.Model(inputs = wideInputs, outputs = wideOutputs)
wideModel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(wideModel.summary())

In [0]:
#Wide Model training
BATCH_SIZE=516
EPOCHS=50
wide = wideModel.fit(x_train,y_train,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.2,verbose=1)

Train on 64 samples, validate on 16 samples
Epoch 1/50
64/64 [==============================] - 1s 10ms/step - loss: 0.6987 - acc: 0.1875 - val_loss: 0.6873 - val_acc: 0.8125
Epoch 2/50
64/64 [==============================] - 0s 154us/step - loss: 0.6823 - acc: 0.9062 - val_loss: 0.6798 - val_acc: 0.8125
Epoch 3/50
64/64 [==============================] - 0s 110us/step - loss: 0.6697 - acc: 0.9062 - val_loss: 0.6721 - val_acc: 0.8125
Epoch 4/50
64/64 [==============================] - 0s 75us/step - loss: 0.6603 - acc: 0.9062 - val_loss: 0.6641 - val_acc: 0.8125
Epoch 5/50
64/64 [==============================] - 0s 112us/step - loss: 0.6516 - acc: 0.9062 - val_loss: 0.6563 - val_acc: 0.8125
Epoch 6/50
64/64 [==============================] - 0s 125us/step - loss: 0.6431 - acc: 0.9062 - val_loss: 0.6485 - val_acc: 0.8125
Epoch 7/50
64/64 [==============================] - 0s 111us/step - loss: 0.6348 - acc: 0.9062 - val_loss: 0.6409 - val_acc: 0.8125
Epoch 8/50
64/64 [================

In [0]:
prediction=wideModel.predict(x_test,verbose=1,batch_size=BATCH_SIZE)

***Deep CNN Model Implementation***

In [0]:
BETA=64
GAMMA=10

deepmodel = keras.models.Sequential()
deepmodel.add(layers.Reshape((147,7),input_shape=(1029,)))
deepmodel.add(layers.Conv1D(7,7,activation='relu',input_shape=(147,7)))
deepmodel.add(layers.MaxPool1D(7))
deepmodel.add(layers.Dense(BETA,activation='relu'))
deepmodel.add(layers.Flatten())
deepmodel.add(layers.Dense(1,activation='sigmoid'))
deepmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(deepmodel.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 147, 7)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 141, 7)            350       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 20, 7)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 20, 64)            512       
_________________________________________________________________
flatten_2 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 1281      
Total params: 2,143
Trainable params: 2,143
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
deepmodel.fit(x_train,y_train,epochs=EPOCHS,validation_split=0.2,batch_size=1029,verbose=1)

In [0]:
print(wideModel.input,"",deepmodel.input)

Tensor("input_16:0", shape=(?, 1029), dtype=float32)  Tensor("reshape_18_input:0", shape=(?, 1029), dtype=float32)


In [0]:
mergedOutput = layers.concatenate([wideModel.output,deepmodel.output])
mergedOutput = layers.Dense(1,activation='sigmoid')(mergedOutput)

combinedModel = keras.Model(wideModel.input + [deepmodel.input], mergedOutput)
print(combineModel.summary())